### OCNC 2018 Introductory Session for Biologists:
# Numerical Methods for Differential Equations
2018.6.25 by Kenji Doya

## Contents
* What is a differential equation
* Euler method
* `ode()` fuction in `scipy`
* Stability and eigenvalus
* Hodgkin-Huxley neuron model


* Partial differential equations
    * cable equation
    * Hodgkin-Huxley axon

## References
* Stephen Wiggins: Introduction to Applied Nonlinear Dynamical Systems and Chaos, 2nd ed., Springer (2003).
* [Scipy Lecture Notes](http://www.scipy-lectures.org): Section 1.5.7 Numerical Integration

## What is a differential equation
A *differential equation* is an equation that includes a derivative $df(x)/dx$ of a function $f(x)$.  

If the independent variable $x$ is single, such as time, it is called an *ordinary differential equation (ODE)*.  

If there are multiple independent variables, such as space and time, the equation includes *partial derivatives* and called a *partial differential equation (PDE)*.

Here we consider ODEs of the form
$$ dy/dt = f(y, t) $$
which describes the temporal dynamics of a varibale $y$ over time $t$.  It is also called a *continuous-time dynamical system*.

Finding the variable $y$ as an explicit function of time $y(t)$ is called *solving* or *integrating* the ODE.  
When it is done numerically, it is aslo called *simulating*.



## Analytic Solutions
Solving a differential equation is an inverse problem of differentiation, for which analytic solution may not be available.  

The simplest case where analytic solutions are available is *linear* differential equations 
$$ dy/dt = A y $$
where $y$ is a real variable or a real vector, and $A$ is a constant coefficient or matrix.  

### Linear ODEs
In general, a differential equation can have multiple solutions. 

For example, for a scalar linear ODE
$$ dy/dt = a y, $$
the solution is given by
$$ y(t) = C e^{at}, $$
where $C$ can be any real value.

When the value of $y$ at a certain time is specified, the solution becomes unique.  
For example, by specifying $y(0)=3$, from $e^{a0}=e^0=1$, we have $C=3$ and a particular solution $y(t)=3e^{at}$.

For a second-order linear ODE
$$ d^2y/dt^2 = -a^2 y, $$ 
the solution is given by
$$ y(t) = C_1 \sin at + C_2 \cos at $$
where $C_1$ and $C_2$ are determined by spedifying $y$ and $dy/dt$ at certain time.

### Analytically solvable ODEs
Other cases where analytic solutions are well known are:
* Time-varying linear: $dy/dt = g(t)y + h(t)$
* Separable: $dy/dt = g(t)/h(y(t))$
* Other cases that can be reduced to above by change of variables, etc.

## Euler Method
The most basic way of sovling an ODE numerically is *Euler Method*.  
For an ODE
$$ dy/dt = f(y,t) $$
with an ititial condition $y(0)=y_0$, the solution is iteratively approximated by
$$ y(t+\Delta t) = y(t) + f(y,t) \Delta t $$
with a small time step $\Delta t$.

In [ ]:
# As usual, import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def euler(f, y0, dt, n, *args):
    """f: righthand side of ODE dy/dt=f(y,t)
        y0: initial condition y(0)=y0
        dt: time step
        n: iteratons
        args: parameter for f(y,t,*args)"""
    d = np.array([y0]).size  ## state dimension
    y = np.zeros((n+1, d))
    y[0] = y0
    t = 0
    for k in range(n):
        y[k+1] = y[k] + f(y[k], t, *args)*dt
        t = t + dt
    return(y)

Let us test this with a first-order linear ODE.

In [ ]:
def first(y, t, a):
    """first-order linear ODE dy/dt = a*y"""
    return(a*y)

In [ ]:
y = euler(first, 1, 0.1, 100, 1)
plt.plot(y)

A second-order ODE 
$$ d^2y/dt^2 = a_2 dy/dt + a_1 y + a_0 $$
can be converted to a first-order ODE with a 2-dimensional state vector $(y_1, y_2) = (y, dy/dt)$ as 
$$ dy_1/dt = y_2 $$
$$ dy_2/dt = a_2 y_2 + a_1 y_1 + a_0 $$

In [ ]:
def second(y, t, a):
    """second-order linear ODE """
    y1, y2 = y
    return(np.array([y2, a[2]*y2 + a[1]*y1 + a[0]]))

In [ ]:
y = euler(second, [1, 0], 0.01, 1000, [0, -1, 0])
plt.plot(y)

Let us see how the time step affects the accuracy of the solution.

In [ ]:
steps = [0.01, 0.03, 0.1, 0.3, 1]
tend = 5
a = -1.5
for dt in steps:
    y = euler(first, 1, dt, int(tend/dt), a)
    plt.plot(np.linspace(0, tend, len(y)), y)

## Scipy's Integrate package
For any serious integration, it is better to use a well tested and proven library, such as `odeint()` in `scipy`.

In [ ]:
from scipy.integrate import odeint

In [ ]:
help(odeint)

`odeint()` internally uses adaptive time steps, and returns values of `y` for time points specified in `t` by interpolation.

Try with the first order linear equaiton.

In [ ]:
t = np.arange(0, 10, 0.1)  # time points
y = odeint(first, 1, t, args=(1,))
plt.plot(t, y, '+-')

In [ ]:
# If you are interested in details of computing
y, info = odeint(first, 1, t, args=(-1,), full_output=1)
plt.plot(t, y, '+-')
plt.plot(info['tcur'], np.zeros_like(info['tcur']), '+')
print(info['mused'])

And the second order.

In [ ]:
t = np.arange(0,10,0.1)  # time points
y = odeint(second, [1, 0], t, args=([0.1, -1, -1],))
plt.plot(t, y, '+-')

## Fixed Point and Stability
A point $y$ where $dy/dt=f(y)=0$ is called a *fixed point*.

A fixed point is characterized by its *stability*:
* Stable
    - Attractor
    ![attractor](figures/attractor.png)
    - Neutrally stable
    <img src="figures/neutral.png" width="250">
* Unstable
    - Saddle
    <img src="figures/saddle.png" width="250">
    - Repellor
    ![repellor](figures/repellor.png)
    
For a linear dynamical system
$$ dy/dt = A y $$
where $y$ is an $n$ dimensional vector and $A$ is an $n\times n$ matrix, the origin $y=0$ is a fixed point. 
Its stability is determined by the eigenvalues of $A$.

## Linear differential equation system

Here we take a vector-matrix notation
$$ d{\bf y}/dt = A {\bf y} $$
where 
$$ {\bf y}=\pmatrix{y_0 \\ y_1}
    \quad
    A = \pmatrix{a & b \\ c & d} $$

In [ ]:
def linear(y, t, A):
    """Linear dynamcal system dy/dt = Ay
    y: n-dimensional state vector
    t: time (not used, for compatibility with odeint())
    A: n*n matrix"""
    # y is an array (row vector), A is a matrix
    return(np.dot(A, y))

In [ ]:
A = np.array([[0, 1], [-1, 0]])

In [ ]:
y0 = np.array([1, -0.001])
t = np.arange(0, 10, 0.1)
y = odeint(linear, y0, t, args=(A,))
plt.plot(y[0,0], y[0,1], 'o')   # starting point
plt.plot(y[:,0], y[:,1], '+-')  # trajectory
plt.axis('equal');

### Eigenvalues and eigenvectors
The behavior of the linear differential equation is determined by the *eigenvalues* and *eigenvectors* of the coefficient matrix $A$.

With a matrix multiplication, a vector $\bf{x}$ is mapped to $A\bf{x}$, which can chenge the direction and size of the vector.  
An *eigenvector* of $A$ is a vector that keeps its direction after multiplication and its scaling coefficient is called the *eigenvalue*.  

Eigenvalues and eigenvectors are derived by solving the equation
$$ A\bf{x} = \lambda \bf{x}. $$

For the 2x2 matrix $A = \pmatrix{a & b \\ c & d}$, the eivenvalues are given from 
$$ \det (A - \lambda I) = (a-\lambda)(d-\lambda) - bc = 0 $$
as
$$ \lambda = \frac{a+d \pm \sqrt{(a-d)^2 + 4bc}}{2} $$

Complex eigenvalues makes an oscillatory solution.  
The signs of the real part determines the stability.

You can check eigenvalues and eigenvectors by `linalg.eig()` function.

In [ ]:
np.linalg.eig(A)

Try different settings of A and corresponding solutions.

In [ ]:
A = np.array([[-1, 1], [-1, 0]])
np.linalg.eig(A)

In [ ]:
y0 = np.array([1, -0.001])
t = np.arange(0, 10, 0.1)
y = odeint(linear, y0, t, args=(A,))
plt.plot(y[0,0], y[0,1], 'o')   # starting point
plt.plot(y[:,0], y[:,1], '+-')  # trajectory
plt.axis('equal');

In [ ]:
A = np.array([[1, 1], [-1, 0]])
np.linalg.eig(A)

In [ ]:
A = np.array([[-1, 0], [0, 1]])
np.linalg.eig(A)

## Time varying input

In [ ]:
def stim(t):
    return (1<t)*(t<2)*10

Alpha function for simulating EPSP

In [ ]:
def alpha(y, t):
    tau = 2
    return np.array([ stim(t)-y[0], y[0]-y[1]])/tau
t = np.arange(0, 10, 0.1)
y = odeint(alpha, [0,0], t)
plt.plot(t, stim(t))
plt.plot(t, y)
plt.xlabel("time");

## Hodgkin-Huxley neuron models
The Hodgkin-Huxley (HH) model considers a neuron as an electric circuit as depicted below.
![HH model](figures/hhmodel.jpg)

On the cellular membrane, there are *ionic channels* that pass specific type of ions. Sodium ions (Na$^+$) are scarce inside the cell, so that when sodium channel opens, positive charges flood into the cell to cause excitation. Potassium ions (K$^+$) are rich inside the cell, so that when potassium channel opens, positive charges flood out of the cell to cause inhibition. The HH model assumes a 'leak' current that put together all other ionic currents.

The ingeniety of Hodgkin and Huxley is that they inferred from careful data analysis that a single sodium channel consists of three *activation* gates and one *inactivation* gate, and a single potassium channel consists of four activation gates. Such structures were later confirmed by genomics and imaging.

The electric potential inside the neuron $V$ follows the following equation:

$$C \frac{dV}{dt} = g_{Na}m^3h(E_{Na}-V) + g_Kn^4(E_K-V) + g_L(E_L-V) + I$$

Here, $m$, $h$, and $n$ represent the proportions of opening of sodium activation, sodium inactivation, and potassium activation gates, respectively. 
They follow the following differential equations with their rates of opening and closing, $\alpha(V)$ and $\beta(V)$, depending on the membrane voltage $V$.

$$\frac{dm}{dt} = \alpha_m(V)(1-m) - \beta_m(V)m$$

$$\frac{dh}{dt} = \alpha_h(V)(1-h) - \beta_h(V)h$$

$$\frac{dn}{dt} = \alpha_n(V)(1-n) - \beta_n(V)n$$

These compose a system of four-dimensional non-linear differential equations. Another amazing thing about Hodgkin and Huxley is that they could simulate the solutions of these differential equations by a hand-powered computer.

Below is a code to simulate the HH model by Python. Much easier!

In [ ]:
def hh(y, t):
    # HH: Hodgkin-Huxley (1952) model
    # input current (uA/cm^2)
    Ie = stim(t)
    # membrane capacitance (uF/cm^2)
    Cm = 1
    # maximum conductances (uS/cm^2)
    gna = 120  # sodium
    gk = 36    # potassium
    gl = 0.3   # leak
    # reversal potentials (mV)
    Ena = 50   # sodium
    Ek = -77   # potassium
    El = -54.4 # leak
    # potential and activation/inactivation (for readability)
    v, m, h, n = y
    # sodium current activation
    am = 0.1*(v+40)/(1-np.exp(-(v+40)/10))
    bm = 4*np.exp(-(v+65)/18);
    # sodium current inactivation
    ah = 0.07*np.exp(-(v+65)/20)
    bh = 1/(1+np.exp(-(v+35)/10))
    # potassium current activation
    an = 0.01*(v+55)/(1-np.exp(-(v+55)/10))
    bn = 0.125*np.exp(-(v+65)/80)
    # time derivative
    return [ (Ie - gna*m**3*h*(v-Ena) - gk*n**4*(v-Ek) - gl*(v-El))/Cm,
            am*(1-m)-bm*m, ah*(1-h)-bh*h, an*(1-n)-bn*n];

In [ ]:
def stim(t):
    return 0.1*t  # ramp current

In [ ]:
tt = np.arange(0,200,0.1)
y0 = [ -75, 0.1, 0.6, 0.3]
yt = odeint(hh, y0, tt)
plt.plot(tt,stim(tt))
plt.plot(tt,yt);

Subplot for better scaling

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(tt, stim(tt));  # stim
plt.plot(tt, yt[:,0]);   # Ie, V
plt.legend(("I(t)", "V(t)"), loc='upper left');
plt.ylabel("I, V");
plt.subplot(2, 1, 2)
plt.plot(tt, yt[:,1:]);  # m, h, n
plt.legend(("m(t)", "h(t)", "n(t)"), loc='upper left')
plt.ylabel("m, h, n");
plt.xlabel("time (ms)");

Phase space

In [ ]:
plt.plot(yt[:,0],yt[:,2])
plt.xlabel("V")
plt.ylabel("h");

## Partial Differential Equations

### Cable equation
![cable](figures/cable.png)

In [ ]:
def cable(v, t):
    # CABLE: linear cable equation
    # specific parameters
    Ri = 200e-6   # intracellular resistance (Mohm*cm)
    Rm = 0.05     # membrane resistance (Mohm*cm^2)
    Cm = 1.0        # membrane capacitance (uF/cm^2)
    # parameters per length
    d = 0.1e-4        # cable diameter (cm)
    ra = 4*Ri/(np.pi*d**2)  # axial resistance (Mohm/cm)
    rm = Rm/(np.pi*d) # membrane resistance (Mohm*cm)
    cm = Cm*np.pi*d   # axial resistance (uF/cm)
    # discretization
    dx = 10e-4    # segment length (cm)
    N = len(v)    # number of segments
    # input current to the center segment
    Ie = np.zeros(N)
    Ie[np.int(N/2)] = stim(t)  # (uA)
    # second-order spatial derivative
    # open (Dirichlet) boundary for 0
    # closed (Neumann) boundary for N-1
    # by for loop:
    vxx = np.zeros(N)
    for i in range(N):
        vxx[i] = ((v[i+1] if i<N-1 else v[N-1]) + (v[i-1] if i>0 else 0) - 2*v[i])/dx**2
    # by vector calculation:
    # vxx = (np.append(v[1:],v[-1]) + np.insert(v[:-1],0,0) - 2*v)/dx**2   return (Ie + yxx/ra - y/rm)/cm
    return (Ie + vxx/ra - v/rm)/cm
    # return (Ie + d/(4*Ri)*vxx - v/Rm)/Cm

In [ ]:
cable(np.arange(-1,1,0.1), 0)

In [ ]:
def stim(t):
    return (0.2<t)*(t<1)*1
tt = np.arange(0, 2, 0.01)
y0 = np.zeros(51)
yt = odeint(cable, y0, tt)
plt.plot(tt,stim(tt))
plt.plot(tt,yt)
plt.xlabel("t");

Spatial plot

In [ ]:
plt.plot(yt.T)
plt.xlabel("x");

In [ ]:
plt.imshow(yt.T)
plt.xlabel("t");
plt.ylabel("x");

## Hodgkin-Huxley Axon

In [ ]:
def axon(y, t):
    # AXON: Hodgkin-Huxley axon
    # y = [v, m, h, n]
    N = np.int(len(y)/4)   # number of segments
    v = y[0:N]     # membrane potential (mV)
    m = y[N:2*N]   # sodium current activation
    h = y[2*N:3*N] # sodium current inactivation
    n = y[3*N:4*N] # potassium current activation
    # Cable Equation
    # specific parameters
    Ri = 200e-6   # intracellular resistance (Mohm*cm)
    # Rm = 0.05     # membrane resistance (Mohm*cm^2)
    Cm = 1        # membrane capacitance (uF/cm^2)
    # parameters per length
    d = 0.1e-4        # cable diameter (cm)
    ra = 4*Ri/(np.pi*d**2)  # axial resistance (Mohm/cm)
    #rm = Rm/(np.pi*d) # membrane resistance (Mohm*cm)
    cm = Cm*np.pi*d   # axial resistance (uF/cm)
    # discretization
    dx = 1e-1      # segment length (cm)
    # input current to the end segment
    Ie = np.zeros(N)
    Ie[0] = stim(t)  # (uA/cm^2)
    # second-order spatial derivative with Neumann boundary
    # by for loop:
    vxx = np.zeros(N)
    #for i in range(N):
    #    vxx[i] = ((v[N-1] if i==N-1 else v[i+1]) + (v[0] if i==0 else v[i-1]) - 2*v[i])/dx**2
    # by vector calculation:
    vxx = (np.append(v[1:],v[-1]) + np.insert(v[:-1],0,v[0]) - 2*v)/dx**2
    #
    # HH Model
    # maximum conductances (uS/cm^2)
    gna = 120  # sodium
    gk = 36    # potassium
    gl = 0.3   # leak
    # reversal potentials (mV)
    Ena = 50   # sodium
    Ek = -77   # potassium
    El = -54.4 # leak
    # activation and inactivation vectors
    am = 0.1*(v+40)/(1-np.exp(-(v+40)/10))
    bm = 4*np.exp(-(v+65)/18)
    ah = 0.07*np.exp(-(v+65)/20)
    bh = 1/(1+np.exp(-(v+35)/10))
    an = 0.01*(v+55)/(1-np.exp(-(v+55)/10))
    bn = 0.125*np.exp(-(v+65)/80)
    # time derivative
    vdot = (Ie + d/(4*Ri)*vxx - gna*m**3*h*(v-Ena) - gk*n**4*(v-Ek) - gl*(v-El))/Cm
    return np.concatenate((vdot, am*(1-m)-bm*m, ah*(1-h)-bh*h, an*(1-n)-bn*n))

In [ ]:
def stim(t):
    return (1<t)*(t<2)*50
N = 30  # number of compartments
tt = np.arange(0,10,0.01)
y0 = np.repeat( np.array([-65, 0.1, 0.6, 0.3]), N)
yt = odeint(axon, y0, tt)
plt.subplot(5,1,1)
plt.plot(tt,yt[:,0:N])   # V
plt.subplot(5,1,2)
plt.plot(tt,yt[:,N:2*N])  # m
plt.subplot(5,1,3)
plt.plot(tt,yt[:,2*N:3*N])  # h
plt.subplot(5,1,4)
plt.plot(tt,yt[:,3*N:4*N])  # n
plt.subplot(5,1,5)
plt.plot(tt,stim(tt))  # stim
plt.xlabel("time (ms)");

In [ ]:
plt.imshow(yt[:, 0:N].T, aspect='auto')  # take Vs
plt.colorbar();
plt.xlabel("time (ms)");